In [2]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install wget

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Download the library from the following link:
# https://artint.info/AIPython/

from os.path import exists
from pathlib import Path
from typing import Final
from zipfile import ZipFile

from wget import download

URL: Final[str] = 'https://artint.info/AIPython/aipython.zip'
DESTINATION: Final[Path] = Path('aipython.zip')

if not exists(DESTINATION):
    download(url=URL, out=DESTINATION.absolute().as_posix())

if not exists(DESTINATION.stem):
    with ZipFile(DESTINATION, 'r') as zip_ref:
        zip_ref.extractall(DESTINATION.stem)

In [4]:
import sys

# Cheat so I don't have to make `aipython` a package with setuptools or move main.ipynb into the `aipython` directory
sys.path.append(DESTINATION.stem)

In [5]:
import pandas as pd

# Makes it easier to transfer data to .tex tables
data: pd.DataFrame = pd.DataFrame(
    columns=['problem', 'used_heuristic', 'elapsed_time']
)

In [6]:
#;; Air cargo transport problem.
#;; http://www.inf.unibz.it/~tessaris/teaching/AI_06-07/labs/2007-01-10/planning.html
#;;

#(define (domain air-cargo)
#  (:requirements :strips)
#  (:predicates (In ?obj ?place)
#           (At ?obj ?place)
#           (Cargo ?obj)
#           (Plane ?obj)
#           (Airport ?obj))

#  (:action LOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (At ?c ?a) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (In ?c ?p) (not (At ?c ?a))))
#  
#  (:action UNLOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (In ?c ?p) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (At ?c ?a) (not (In ?c ?p))))
#  
#  (:action FLY
#     :parameters (?p ?from ?to)
#     :precondition (and (At ?p ?from)
#             (Plane ?p) (Airport ?from) (Airport ?to))
#     :effect (and (At ?p ?to) (not (At ?p ?from))))
#  )

In [7]:
# STRIPS domain - Air Cargo Transportation
from stripsProblem import Strips, STRIPS_domain, Planning_problem

def In(obj: str) -> str:
    return obj + '_is_in'

def At(obj: str) -> str:
    return obj + '_is_at'

def Cargo(obj: str) -> str:
    return obj + '_is_cargo'

def Plane(obj: str) -> str:
    return obj + '_is_plane'

def Airport(obj: str) -> str:
    return obj + '_is_airport'

def Load(cargo: str, plane: str, airport: str) -> str:
    return 'load_' + cargo + '_from_' + airport + '_to_' + plane

def Unload(cargo: str, plane: str, airport: str) -> str:
    return 'unload_' + cargo + '_from_' + plane + '_to_' + airport

def Fly(plane: str, airport_from: str, airport_to: str) -> str:
    return 'fly_' + plane + '_from_' + airport_from + '_to_' + airport_to


null: set[str] = {'null'}
def create_air_cargo_transportation(cargo: set[str], planes: set[str], airports: set[str], cargo_forbidden_airports: dict[str, set[str]], planes_forbidden_airports: dict[str, set[str]]) -> STRIPS_domain:
    assert len(cargo) > 0, "Cargo set must not be empty."
    assert len(planes) > 0, "Planes set must not be empty."
    assert len(airports) > 0, "Airports set must not be empty."
    
    assert all(c in cargo_forbidden_airports for c in cargo), "All cargo must have a forbidden airports set that may or may not be empty."
    assert all(p in planes_forbidden_airports for p in planes), "All planes must have a forbidden airports set that may or may not be empty."
    
    #feature_domain_dict = {Cargo(x):True for x in cargo}
    #feature_domain_dict.update({Cargo(x):False for x in cargo_storage})
    #feature_domain_dict.update({Plane(x):True for x in planes})
    #feature_domain_dict.update({Plane(x):False for x in cargo|airports})
    #feature_domain_dict.update({Airport(x):True for x in airports})
    #feature_domain_dict.update({Airport(x):False for x in cargo|planes})
    feature_domain_dict: dict[str, set[str]] = {In(x): planes | null
                                                for x in cargo}
    feature_domain_dict.update({At(x): (airports - (cargo_forbidden_airports | planes_forbidden_airports)[x]) | null
                                for x in planes | cargo})


    state_map: set[Strips] = {Strips(Load(c, p, a), {At(c): a, At(p): a}, {In(c): p, At(c): 'null'})
                              for c in cargo
                              for p in planes
                              for a in airports}

    state_map.update({Strips(Unload(c, p, a), {In(c): p, At(p): a}, {At(c): a, In(c): 'null'})
                      for c in cargo
                      for p in planes
                      for a in airports
                      if a not in cargo_forbidden_airports[c]})

    state_map.update({Strips(Fly(p, a_from, a_to), {At(p): a_from}, {At(p): a_to})
                      for p in planes
                      for a_from in airports
                      for a_to in airports
                      if a_from != a_to and a_to not in planes_forbidden_airports[p]})

    return STRIPS_domain(feature_domain_dict, state_map)

In [8]:
# Domain
air_cargo1dom: STRIPS_domain = create_air_cargo_transportation(
    cargo={'c1', 'c2', 'c3', 'c4', 'c5'},
    planes={'p1', 'p2',},
    airports={'sfo', 'jfk', 'lax',},
    cargo_forbidden_airports={'c1': set(), 'c2': set(), 'c3': set(), 'c4': set(), 'c5': set(),},
    planes_forbidden_airports={'p1': {'lax'}, 'p2': set(),},
)

# Initial state
air_setup1: dict[str, str] = {
    At('c1'): 'sfo',
    At('c2'): 'jfk',
    At('c3'): 'lax',
    At('c4'): 'jfk',
    At('c5'): 'lax',
    
    At('p1'): 'sfo',
    At('p2'): 'jfk',
    
    # Airplanes start loaded with cargo to make the problem easier
    In('c1'): 'p1',
    In('c2'): 'p2',
    In('c3'): 'null',
    In('c4'): 'null',
    In('c5'): 'null',
}

# Goal state
air_goal1: dict[str, str] = {
    At('c1'): 'jfk',
    At('c2'): 'sfo',
    At('c3'): 'sfo',
    At('c4'): 'lax',
    At('c5'): 'sfo',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
    In('c4'): 'null',
    In('c5'): 'null',
}

air_cargo1: Planning_problem = Planning_problem(air_cargo1dom, air_setup1, air_goal1)

In [9]:
from time import perf_counter

from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

# Problem 1
# Problem idea: We have many more cargo to send than planes, and the planes are not allowed to go to certain airports.
print('Problem 1 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1', False, end - start]

Problem 1 - No heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'jfk', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'null', 'c5_is_in': 'null'}
   --load_c4_from_jfk_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'null', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'p2', 'c5_is_in': 'null'}
   --fly_p2_from_jfk_to_lax--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'null', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'p2', 'c5_is_in': 'null'}
   --load_c5_from_lax_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'null', 'c5_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in':

In [10]:
def h_unloaded_cargo(assignment, goal):
    """This heuristic counts the number of cargo that are not yet at their goal location. The idea is that the more cargo that are not at their goal, the further away we are from the goal state."""
    return sum(1 for k, v in goal.items() if assignment[k] != v and '_is_in' in k)

In [11]:
# Solution idea - we solve the problem by using a heuristic that counts the number of cargo that are not yet at their goal location, speeding the computation.
print('Problem 1 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1, h_unloaded_cargo)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1', True, end - start]

Problem 1 - With heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'jfk', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'null', 'c5_is_in': 'null'}
   --load_c4_from_jfk_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'null', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'p2', 'c5_is_in': 'null'}
   --fly_p2_from_jfk_to_lax--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'null', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'p2', 'c5_is_in': 'null'}
   --load_c5_from_lax_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'null', 'c5_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in

In [12]:
# Domain
air_cargo2dom: STRIPS_domain = create_air_cargo_transportation(
    cargo={'c1', 'c2', 'c3'},
    planes={'p1', 'p2', 'p3'},
    airports={'sfo', 'jfk', 'lax', 'ord', 'atl',},
    cargo_forbidden_airports={'c1': set(), 'c2': set(), 'c3': set(),},
    planes_forbidden_airports={'p1': {'ord'}, 'p2': {'lax'}, 'p3': set(),},
)

# Initial state
air_setup2: dict[str, str] = {
    At('c1'): 'atl',
    At('c2'): 'ord',
    At('c3'): 'jfk',

    At('p1'): 'sfo',
    At('p2'): 'jfk',
    At('p3'): 'ord',
    
    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
}

# Goal state
air_goal2: dict[str, str] = {
    At('c1'): 'jfk',
    At('c2'): 'sfo',
    At('c3'): 'sfo',
    
    At('p3'): 'jfk',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
}

air_cargo2: Planning_problem = Planning_problem(air_cargo2dom, air_setup2, air_goal2)

In [13]:
# Problem 2
# Problem idea: We have small amount of cargo, but many airports and planes, some planes must end in specific airports.
print("Problem 2 - No heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo2)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2', False, end - start]

Problem 2 - No heuristic:
Solution: {'c1_is_at': 'atl', 'c2_is_at': 'ord', 'c3_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'ord', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --load_c3_from_jfk_to_p2--> {'c1_is_at': 'atl', 'c2_is_at': 'ord', 'c3_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'ord', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'p2'}
   --fly_p2_from_jfk_to_ord--> {'c1_is_at': 'atl', 'c2_is_at': 'ord', 'c3_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'ord', 'p3_is_at': 'ord', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'p2'}
   --fly_p3_from_ord_to_atl--> {'c1_is_at': 'atl', 'c2_is_at': 'ord', 'c3_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'ord', 'p3_is_at': 'atl', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'p2'}
   --load_c2_from_ord_to_p2--> {'c1_is_at': 'atl', 'c2_is_at': 'null', 'c3_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'ord', 'p3_is_at': 'atl', 'c1_is_in': 'null', 'c2_is

In [14]:
def h_mismatched_planes(assignment, goal):
    """This heuristic counts the number of planes that are not yet at their goal location. The idea is that the more planes that are not at their goal, the further away we are from the goal state."""
    return sum(1 for k, v in goal.items() if assignment[k] != v and '_is_at' in k and 'p' in k)

In [15]:
# Solution idea - hopefully making so that free planes fly to their desired location, and in the meantime planes that are not restricted to be on specific airports make the cargo moving....
print("Problem 2 - With heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo2, h_mismatched_planes)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2', True, end - start]

Problem 2 - With heuristic:
Solution: {'c1_is_at': 'atl', 'c2_is_at': 'ord', 'c3_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'ord', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --load_c2_from_ord_to_p3--> {'c1_is_at': 'atl', 'c2_is_at': 'null', 'c3_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'ord', 'c1_is_in': 'null', 'c2_is_in': 'p3', 'c3_is_in': 'null'}
   --fly_p3_from_ord_to_jfk--> {'c1_is_at': 'atl', 'c2_is_at': 'null', 'c3_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p3', 'c3_is_in': 'null'}
   --load_c3_from_jfk_to_p3--> {'c1_is_at': 'atl', 'c2_is_at': 'null', 'c3_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p3', 'c3_is_in': 'p3'}
   --fly_p3_from_jfk_to_atl--> {'c1_is_at': 'atl', 'c2_is_at': 'null', 'c3_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'p3_is_at': 'atl', 'c1_is_in': 'null', 'c2_i

In [16]:
# Domain
air_cargo3dom: STRIPS_domain = create_air_cargo_transportation(
    cargo={'c1', 'c2', 'c3', 'c4'},
    planes={'p1', 'p2', 'p3',},
    airports={'sfo', 'jfk', 'lax',},
    cargo_forbidden_airports={'c1': {'lax'}, 'c2': {'jfk'}, 'c3': {'lax'}, 'c4': {'sfo'},},
    planes_forbidden_airports={'p1': {'lax'}, 'p2': {'jfk'}, 'p3': {'sfo'},}
)

# Initial state
air_setup3: dict[str, str] = {
    At('c1'): 'sfo',
    At('c2'): 'lax',
    At('c3'): 'jfk',
    At('c4'): 'jfk',

    At('p1'): 'sfo',
    At('p2'): 'lax',
    At('p3'): 'jfk',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
    In('c4'): 'null',
}

# Goal state
air_goal3: dict[str, str] = {
    At('c1'): 'jfk',
    At('c2'): 'sfo',
    At('c3'): 'sfo',
    At('c4'): 'lax',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
    In('c4'): 'null',
}

air_cargo3: Planning_problem = Planning_problem(air_cargo3dom, air_setup3, air_goal3)

In [17]:
# Problem 3
# Problem idea: Medium amount of cargo, medium amount of planes, and some cargo and planes are not allowed to go to certain airports. And some goods are not allowed in some airports.
print("Problem 3 - No heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo3)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3', False, end - start]

Problem 3 - No heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'null'}
   --load_c1_from_sfo_to_p1--> {'c1_is_at': 'null', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'null'}
   --load_c4_from_jfk_to_p3--> {'c1_is_at': 'null', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'p3'}
   --fly_p1_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'null', 'p1_is_at': 'jfk', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'p3'}
   --fly_p3_from_jfk

In [18]:
def h_combined(assignment, goal):
    """This heuristic combines the previous two heuristics."""
    return h_unloaded_cargo(assignment, goal) + h_mismatched_planes(assignment, goal)

In [19]:
# Solution idea - combine all the previous heuristics to make the computation faster.
print("Problem 3 - With heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo3, h_combined)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3', True, end - start]

Problem 3 - With heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'null'}
   --load_c1_from_sfo_to_p1--> {'c1_is_at': 'null', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'null'}
   --fly_p1_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at': 'lax', 'c3_is_at': 'jfk', 'c4_is_at': 'jfk', 'p1_is_at': 'jfk', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null', 'c3_is_in': 'null', 'c4_is_in': 'null'}
   --load_c3_from_jfk_to_p1--> {'c1_is_at': 'null', 'c2_is_at': 'lax', 'c3_is_at': 'null', 'c4_is_at': 'jfk', 'p1_is_at': 'jfk', 'p2_is_at': 'lax', 'p3_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null', 'c3_is_in': 'p1', 'c4_is_in': 'null'}
   --load_c2_from

In [23]:
# Sub goal / initial state
air_sub_problem_1: dict[str, str] = {
    At('c1'): 'jfk',
    At('c2'): 'sfo',
    At('c3'): 'sfo',
    At('c4'): 'jfk',
    At('c5'): 'lax',
    
    At('p1'): 'jfk',
    At('p2'): 'sfo',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
    In('c4'): 'null',
    In('c5'): 'null',
}

air_cargo1_sub_problem_1: Planning_problem = Planning_problem(air_cargo1dom, air_setup1, air_sub_problem_1)
air_cargo1_sub_problem_2: Planning_problem = Planning_problem(air_cargo1dom, air_sub_problem_1, air_goal1)

In [24]:
# Sub problem 1
# Idea: finish delivering the initially loaded cargo and c3 to specific airports

print('Problem 1 - Sub Problem 1 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1_sub_problem_1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1 - Sub Problem 1', False, end - start]

Problem 1 - Sub Problem 1 - No heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'jfk', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'null', 'c5_is_in': 'null'}
   --fly_p2_from_jfk_to_lax--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'c4_is_at': 'jfk', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'null', 'c4_is_in': 'null', 'c5_is_in': 'null'}
   --load_c3_from_lax_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'null', 'c4_is_at': 'jfk', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'lax', 'c1_is_in': 'p1', 'c2_is_in': 'p2', 'c3_is_in': 'p2', 'c4_is_in': 'null', 'c5_is_in': 'null'}
   --fly_p2_from_lax_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'null', 'c4_is_at': 'jfk', 'c5_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p1', 'c2_is_in': '

In [22]:
print('Problem 1 - Sub Problem 2 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1_sub_problem_2)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1 - Sub Problem 2', False, end - start]

Problem 1 - Sub Problem 2 - No heuristic:


KeyError: 'p1_is_at'

In [ ]:
print('Problem 1 - Sub Problem 1 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1_sub_problem_1, h_unloaded_cargo)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1 - Sub Problem 1', True, end - start]

In [ ]:
print('Problem 1 - Sub Problem 2 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1_sub_problem_2, h_unloaded_cargo)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1 - Sub Problem 2', True, end - start]

In [ ]:
# Sub goal / initial state
air_sub_problem_2: dict[str, str] = {
    At('c1'): 'atl',
    At('c2'): 'ord',
    At('c3'): 'jfk',

    At('p3'): 'jfk',
    At('p4'): 'sfo',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
}

air_cargo2_sub_problem_1: Planning_problem = Planning_problem(air_cargo2dom, air_setup2, air_sub_problem_2)
air_cargo2_sub_problem_2: Planning_problem = Planning_problem(air_cargo2dom, air_sub_problem_2, air_goal2)

In [ ]:
# Sub problem 2
# Idea: Ground not needed aircrafts, then solve the problem with less number of planes.

print('Problem 2 - Sub Problem 1 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo2_sub_problem_1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2 - Sub Problem 1', False, end - start]

In [ ]:
print('Problem 2 - Sub Problem 2 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo2_sub_problem_2)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2 - Sub Problem 2', False, end - start]

In [ ]:
print('Problem 2 - Sub Problem 1 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo2_sub_problem_1, h_mismatched_planes)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2 - Sub Problem 1', True, end - start]

In [ ]:
print('Problem 2 - Sub Problem 2 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo2_sub_problem_2, h_mismatched_planes)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2 - Sub Problem 2', True, end - start]

In [ ]:
# Sub goal / initial state
air_sub_problem_3: dict[str, str] = {
    At('c1'): 'jfk',
    At('c2'): 'sfo',
    At('c3'): 'jfk',
    At('c4'): 'jfk',

    In('c1'): 'null',
    In('c2'): 'null',
    In('c3'): 'null',
    In('c4'): 'null',
}

air_cargo3_sub_problem_1: Planning_problem = Planning_problem(air_cargo3dom, air_setup3, air_sub_problem_3)
air_cargo3_sub_problem_2: Planning_problem = Planning_problem(air_cargo3dom, air_sub_problem_3, air_goal3)

In [ ]:
# Sub problem 3
# Idea: Solve the problem one package at a time, to utilize different access of some plane to a given airport.

print('Problem 3 - Sub Problem 1 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo3_sub_problem_1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3 - Sub Problem 1', False, end - start]

In [ ]:
print('Problem 3 - Sub Problem 2 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo3_sub_problem_2)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3 - Sub Problem 2', False, end - start]

In [ ]:
print('Problem 3 - Sub Problem 1 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo3_sub_problem_1, h_combined)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3 - Sub Problem 1', True, end - start]

In [ ]:
print('Problem 3 - Sub Problem 2 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo3_sub_problem_2, h_combined)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3 - Sub Problem 2', True, end - start]

In [ ]:
data

In [ ]:
data.dtypes